# DAVIS 2016 - Load and Inspect Dataset (with Augmentation)

In this notebook we:
- Load the 2016 dataset and create the additional data necessary to train MaskRNN (see below)
- Show sample videos along with the MaskRNN additional data

In [ ]:
import sys
import time
import random
import datasets
import visualize
%matplotlib inline
%load_ext autoreload

## Load
Load the DAVIS 2016 dataset and do all the processing necessary for MaskRNN training. This will take a while if data augmentation is enabled!

In [ ]:
# Configure - *SET* dataset_root to the correct path on your machine!
dataset_root = 'E:/datasets/davis2016/' if sys.platform.startswith("win") else '/media/EDrive/datasets/davis2016/'
train_file = dataset_root + 'ImageSets/480p/train.txt'
options = datasets._DEFAULT_DAVIS16_OPTIONS
options['data_aug'] = True

In [ ]:
# Load data
start = time.time()
dataset = datasets.davis16(train_file, None, dataset_root, options)
end = time.time()
m, s = divmod(end - start, 60)
h, m = divmod(m, 60)
print('Time Taken: {:2.0f}h{:2.0f}m{:2.0f}s for {} videos (includes augmentation time, if any).'.format(h, m, s, dataset.num_videos))

In [ ]:
# Display dataset configuration
dataset.print_config()

## Inspect original dataset

Choose a few videos to inspect (includes data augmented videos) at random.
> Naming convention example: `bear_sc_0.5_fl`, bear video scaled by 0.5 factor and flipped vertically

In [ ]:
subset = 1 # Choose [1|2|3]
random.seed(1)
videos = random.sample(list(dataset.videos.keys()), subset)
print('Chosen videos: {} out of {} videos'.format(videos, dataset.num_videos))

Build a list of individual video frames with:
- video frames overlaid with groundtruth masks (red) and computed mask bounding boxes (blue)
- video frames overlaid with previous-frame masks (green) warped based on the optical flow measured betwwen the previous frame and the current frame, as well as the computed mask bounding box for this warped mask (yellow)
- both combined (useful to see how "off" the optical flow predictions are)

In [ ]:
# Show the individual video frames with groundtruth masks (red) and computed mask bounding boxes (yellow)
frames_with_gt_masks = {}
frames_with_warped_masks = {}
frames_with_both_masks = {}
for video in videos:
    frames_with_gt_masks[video] = dataset.create_frames_with_overlays(video, mode='gt')
    frames_with_warped_masks[video] = dataset.create_frames_with_overlays(video, mode='warped')
    frames_with_both_masks[video] = dataset.create_frames_with_overlays(video, mode='gt_warped')

## Display individual frames

Show the individual video frames overlaid with groundtruth masks (red) and computed mask bounding boxes (blue)

In [ ]:
for video in videos:
    print('{} (groundtruth):'.format(video))
    visualize.display_images(frames_with_gt_masks[video])

Show the overlaid with previous-frame masks (green) warped based on the optical flow measured betwwen the previous frame and the current frame, as well as the computed mask bounding box for this warped mask (yellow)

In [ ]:
for video in videos:
    print('{} (warped masks from flow):'.format(video))
    visualize.display_images(frames_with_warped_masks[video])

Show both combined (useful to see how "off" the optical flow predictions are)

In [ ]:
for video in videos:
    print('{} (groaundtruth masks and warped masks from flow):'.format(video))
    visualize.display_images(frames_with_both_masks[video])

## Display videos

The content of the clips below is the same as above but rather than showing individual frames with their overlays, we re-create the entire video sequence with overlays. The clips help get a better sense of what kind of motion is displacing the warped masks away from their groundtruths.

In [ ]:
video_clip_folder = dataset_root + 'clips/'

### Build the video clips

In [ ]:
# Same as above, but this time turn the images into a MP4 video clip
for video in videos:
    print('{}'.format(video))
    visualize.make_clip(video_clip_folder + video + '_gt.mp4', frames_with_gt_masks[video])
    visualize.make_clip(video_clip_folder + video + '_wrp.mp4', frames_with_warped_masks[video])
    visualize.make_clip(video_clip_folder + video + '_gt_wrp.mp4', frames_with_both_masks[video])

## Display videos

### Groundtruths

In [ ]:
video_clips = [video_clip_folder + videos[video] + '_gt.mp4' for video in range(subset)]

In [ ]:
visualize.show_clips(video_clips)

### Masks warped by optical flow

In [ ]:
video_clips = [video_clip_folder + videos[video] + '_wrp.mp4' for video in range(subset)]

In [ ]:
visualize.show_clips(video_clips)

# Groundtruth masks and masks warped by optical flow

In [ ]:
video_clips = [video_clip_folder + videos[video] + '_gt_wrp.mp4' for video in range(subset)]

In [ ]:
visualize.show_clips(video_clips)